## Introduction to Data Cleaning with the Relational Model

In this exercise, we apply the relational model for data cleaning using DuckDB. This exercise assumes that you have repaired the following data errors in the example City of Chicago Food Inspection data using a tool such as OpenRefine:
* License numbers are all numeric values
* Inspection dates are all valid dates in a consistent format
* City, State, and Zip have been parsed into separate fields

### Learning Objectives

* Apply strategies for data cleaning using the relational model including implementing a relational schema, integrity constraints, and queries.

### Prerequisites

* Install `pandas` and `duckdb` Python libraries
* **Optional**:  Install the [DuckDB command line interface](https://duckdb.org/docs/installation/?version=stable&environment=cli) to use `duckdb` from the terminal.


In [2]:
! pip install pandas duckdb


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import duckdb
import pandas as pd

### Inspect the Data

The cleaned data is in `datasets/inspections/food-inspections-cleaned.csv`. We can use the DuckDB Python API to query the CSV data directly and return a Pandas dataframe:

In [5]:
duckdb.sql("SELECT * FROM '../datasets/inspections/food-inspections-cleaned.csv' limit 5").df()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations
0,2078801,THREE CHEFS RESTURANT,THREE CHEFS RESTURANT,2009471.0,Restaurant,Risk 1 (High),8125 S HALSTED ST,None,IL,60620,2017-08-22,Complaint,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
1,2078380,THE GODDESS GOLD COAST,THE GODDESS AND GROCER,2397687.0,Restaurant,Risk 1 (HIGH),1127 N STATE ST,CCHICAGO,IL,60610,2017-08-14,Complaint,Pass w/ Conditions,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
2,2015423,HAROLD'S CHICKEN 57 LLC.,HAROLD'S CHICKEN,2363519.0,Restaurant,Risk 1 (High),6606 W NORTH AVE,Chicago,IL,60707,2017-07-31,Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
3,2071437,CHINA CAFE,CHINA CAFE,2535883.0,Restaurant,Risk 1 (High),2300-2302 S WENTWORTH AVE BLDG,None,IL,60616,2017-07-26,License,Pass,None
4,2071348,MARKET SELECT,MARKET SELECT,2523569.0,Grocery Store,Risk 3 (Low),912 N ASHLAND AVE,None,IL,60622,2017-07-25,License,Pass,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...


### Create the Schema

The next step is to create a relational schema that can be used to import the dataset.

* Create a file named `schema.sql` that will be used to create a database.
* The file must define the schema for a table named `INSPECTIONS` that will store the inspections data. 
* The table should have a primary key.

In [6]:
with duckdb.connect("inspections.db") as con:
    with open("schema.sql", "r") as f:
        # Create the DB schema
        con.sql(f.read())
        # Import data
        con.sql("INSERT INTO inspections (SELECT * FROM read_csv('../datasets/inspections/food-inspections-cleaned.csv'))")

**Discussion**
* What are the column names and datatypes?
* What can we use for the primary key?

### Query the data

Define a query that can be used to count the number of rows in the `INSPECTIONS` table:

In [7]:
q1 = """
    SELECT COUNT(*) AS COUNT from INSPECTIONS
"""

In [10]:
#test
with duckdb.connect("inspections.db") as con:
    count = con.sql(q1).fetchone()[0]
    assert(count == 720)
    print("Success")

Success


Inspect the contents of the `Violations` column. Note that the column contains multiple nested records delimited with pipe (`|`). 


According to the _First Normal Form_, columns should **not** contain nested values. So our next step is to _normalize_ the violations data. 

But first, we need to understand what each record contains.  Use the DuckDB `regexp_split_to_table()` operation to split each violation into a separate row.

In [13]:
q3 = """
   SELECT inspection_id, regexp_split_to_table(Violations, '\\|') 
   FROM INSPECTIONS
   LIMIT 10
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(q3).df()

df

,inspection_id,"regexp_split_to_table(Violations, '\|')"
0,2078801,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
1,2078801,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATU...
2,2078801,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPE...
3,2078801,29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-4...
4,2078801,32. FOOD AND NON-FOOD CONTACT SURFACES PROPER...
5,2078801,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABE..."
6,2078801,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GO..."
7,2078801,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONST..."
8,2078801,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVI...
9,2078801,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENS...


We can see that the violations have a consistent format:

```
(30). (FOOD IN ORIGINAL CONTAINER, PROPERLY LABELED: CUSTOMER ADVISORY POSTED AS NEEDED) - Comments: (LABEL ALL BULK CONTAINERS IN PREP AREA)
```

The violations refer to a list of standard codes and descriptions found on the [Food Inspection Report Form](https://www.chicago.gov/dam/city/depts/cdph/food_env/general/Food_Protection/Blankinspectionreport.pdf) and consist of:
1. A numeric violation code or identifier
2. A standard description of the violation
3. An optional comment from the inspector at the time of inspection

The next step is to create the new table(s) and use SQL or Python to parse and transform the source data.


**Discussion**
* What should the new table(s) be?
* How should we handle duplicated data such as the violation code and description?

### Transform the Data

Update your `schema.sql` to create additional table(s) to store the violations data:
* `VIOLATIONS`: Table to store unique information about violations. It should have a primary key that serves as a foreign key for the `INSPECTION_VIOLATIONS` table.
* `INSPECTION_VIOLATIONS`: Table to store **unique** information about violations associated with each inspection. It should have a primary key and foreign keys to `INSPECTIONS` and `VIOLATIONS`.

First, we need to write a query that can parse the `Violations` column into its separate parts:
* For each inspection, use `regexp_split_to_table` to split the violations records by the `|` delimiter
* For each violation record, use `regexp_extract` to extract the individual parts (violation code, description, and comment)

In [14]:
q4 = """
    select inspection_id, regexp_extract(
        trim(regexp_split_to_table(inspections.violations, '\\|')),
        '^(\\d+)\\.(.*?)(?:-\\s+Comments?:(.*))?$',
        ['violation_id', 'description', 'comments'], 's'
    ) as violation
    from inspections
"""

with duckdb.connect("inspections.db") as con:
    df = con.sql(q4).df()
df

,inspection_id,violation
0,2078801,"{'violation_id': '2', 'description': ' FACILIT..."
1,2078801,"{'violation_id': '3', 'description': ' POTENTI..."
2,2078801,"{'violation_id': '18', 'description': ' NO EVI..."
3,2078801,"{'violation_id': '29', 'description': ' PREVIO..."
4,2078801,"{'violation_id': '32', 'description': ' FOOD A..."
...,...,...
1878,154225,"{'violation_id': '33', 'description': ' FOOD A..."
1879,154225,"{'violation_id': '38', 'description': ' VENTIL..."
1880,58235,"{'violation_id': '32', 'description': ' FOOD A..."
1881,58235,"{'violation_id': '38', 'description': ' VENTIL..."


This rather complex query uses DuckDB's [text functions](https://duckdb.org/docs/sql/functions/char.html) to parse the column:
* `regexp_split_to_table` splits the delimited string into new rows
* `regexp_extract` uses regular expression capture groups to match parse of the violation string

The regular expression consists of:
* `^(\d+)\.`: The first capture group starts at the beginning of the line (`^`) and consists of one or more numeric digits (`\d+`) followed by a literal period (`\.`)
* `(.*?)`: The second capture group consists of everything between the period and an optional `- Comments:` block
* `(?:-\s+Comments?:(.*))?$'`: The third capture group `(?:...)?` is actually an optional `(...)?` non-capturing group `(?:...)`. This says that `- Comments: ` part is optional, since some violations do not contain a comment.
* `(.*)`: The fourth capture group `(.*)` consists of everything between the `- Comments:` and the end of the line `$`

`regexp_extract` returns a JSON structure that can now be referenced in queries:

```
{'violation_id': '', 'description': '', 'comments': ''}
```

Note that the `s` option to `regexp_extract` specifies that matches are not [newline sensitive](https://duckdb.org/docs/sql/functions/regular_expressions.html#options-for-regular-expression-functions).

In [15]:
q5 = """
    select inspection_id, violation['violation_id'] as violation_id, trim(violation['description']) as description
    from
    (
        select inspection_id, regexp_extract(
            trim(regexp_split_to_table(inspections.violations, '\\|')),
            '^(\\d+)\\.(.*?)(?:-\\s+Comments?:(.*))?$',
            ['violation_id', 'description', 'comments'], 's'
        ) as violation
        from inspections
    ) as inspections
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(q5).df()
df

,inspection_id,violation_id,description
0,2078801,2,FACILITIES TO MAINTAIN PROPER TEMPERATURE
1,2078801,3,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
2,2078801,18,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...
3,2078801,29,PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42-090
4,2078801,32,FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DE...
...,...,...,...
1878,154225,33,FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS C...
1879,154225,38,VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQ...
1880,58235,32,FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DE...
1881,58235,38,VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQ...


We can see here that the violations codes and descriptions are indeed duplicated. The next step is to determine whether they are truly unique.

In [17]:
q5 = """
    select distinct violation['violation_id'] as violation_id, trim(violation['description']) as description
    from
    (
        select regexp_extract(
            trim(regexp_split_to_table(inspections.violations, '\\|')),
            '^(\\d+)\\.(.*?)(?:-\\s+Comments?:(.*))?$',
            ['violation_id', 'description', 'comments'], 's'
        ) as violation
        from inspections
    ) as inspections
    order by cast(violation_id as integer);
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(q5).df()
df

,violation_id,description
0,1,"SOURCE SOUND CONDITION, NO SPOILAGE, FOODS PRO..."
1,2,FACILITIES TO MAINTAIN PROPER TEMPERATURE
2,3,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
3,6,"HANDS WASHED AND CLEANED, GOOD HYGIENIC PRACTI..."
4,7,WASH AND RINSE WATER: CLEAN AND PROPER TEMPERA...
5,8,SANITIZING RINSE FOR EQUIPMENT AND UTENSILS: ...
6,9,"WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRES..."
7,10,"SEWAGE AND WASTE WATER DISPOSAL, NO BACK SIPHO..."
8,11,"ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIG..."
9,12,HAND WASHING FACILITIES: WITH SOAP AND SANITAR...


At this point, we're ready to populate the new `VIOLATIONS` table. If the `violation_id` is specified as the primary key, we will see if these are truly unique codes:

In [18]:
q6 = """
    insert into violations
    select distinct violation['violation_id'] as violation_id, trim(violation['description']) as description
    from
    (
        select regexp_extract(
            trim(regexp_split_to_table(inspections.violations, '\\|')),
            '^(\\d+)\\.(.*?)(?:-\\s+Comments?:(.*))?$',
            ['violation_id', 'description', 'comments'], 's'
        ) as violation
        from inspections
    ) as inspections
    order by cast(violation_id as integer);
"""
with duckdb.connect("inspections.db") as con:
    con.sql(q6)

In [12]:
with duckdb.connect("inspections.db") as con:
    df = con.sql("SELECT * FROM VIOLATIONS").df()
df

,violation_id,description
0,1,"SOURCE SOUND CONDITION, NO SPOILAGE, FOODS PRO..."
1,2,FACILITIES TO MAINTAIN PROPER TEMPERATURE
2,3,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
3,6,"HANDS WASHED AND CLEANED, GOOD HYGIENIC PRACTI..."
4,7,WASH AND RINSE WATER: CLEAN AND PROPER TEMPERA...
5,8,SANITIZING RINSE FOR EQUIPMENT AND UTENSILS: ...
6,9,"WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRES..."
7,10,"SEWAGE AND WASTE WATER DISPOSAL, NO BACK SIPHO..."
8,11,"ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIG..."
9,12,HAND WASHING FACILITIES: WITH SOAP AND SANITAR...


The next step is to populate the `INSPECTIONS_VIOLATIONS` table. This should be easy now, since we already have the query to parse the violations data.

The table should contain the `INSPECTION_ID` referencing the INSPECTIONS table, `VIOLATION_ID` referencing the `VIOLATIONS` table, and inspection-specific `COMMENT`:

In [19]:
q7 = """
    insert into inspection_violations
    select inspection_id, violation['violation_id'] as violation_id, trim(violation['comments']) as comment
    from
    (
        select DISTINCT inspection_id, regexp_extract(
            trim(regexp_split_to_table(inspections.violations, '\\|')),
            '^(\\d+)\\.(.*?)(?:-\\s+Comments?:\\s+(.*))?$',
            ['violation_id', 'description', 'comments'], 's'
        ) as violation
        from inspections 
    ) 
    order by inspection_id, cast(violation_id as integer);
"""
with duckdb.connect("inspections.db") as con:
    con.sql(q7)

In [20]:
with duckdb.connect("inspections.db") as con:
    df = con.sql("SELECT * FROM INSPECTION_VIOLATIONS").df()
df

,inspection_id,violation_id,comment
0,58232,32,All food and non-food contact equipment and ut...
1,58232,38,"Ventilation: All plumbing fixtures, such as to..."
2,58235,32,All food and non-food contact equipment and ut...
3,58235,38,"Ventilation: All plumbing fixtures, such as to..."
4,58236,32,All food and non-food contact equipment and ut...
...,...,...,...
1878,2078801,34,CLEAN FLOORS ALONG WALLS AND IN ALL CORNERS IN...
1879,2078801,35,SEAL ALL OPENINGS IN WALLS ALONG BASEBOARDS IN...
1880,2078801,38,REPAIR LEAKING DRAINPIPE UNDER PREP SINK IN KI...
1881,2078801,40,PROVIDE THERMOMETERS IN ALL COOLERS AND METAL ...


### Integrity Constraints

We can use our knowledge of the domain to define integrity constraints that can be used to further identify errors in the data.  In fact, we have already defined constraints.  

For example, specifing the `violation_id` as the primary key on the violations table indicates that `violation_id -> description`. If this constraint had been violated, our inserts into the table would have failed.

Below we use SQL queries to identify potential errors based on several functional dependencies.

#### Zip -> City, State

Based on our knowledge of US ZIP Codes, we expect that, across all inspection records, the ZIP Code will determine the City and State. We can confirm this with the following query:

In [21]:
c1 = """
    SELECT i1.zip, i1.city, i2.city, i1.state, i2.state 
    FROM inspections i1, inspections i2
    WHERE i1.zip = i2.zip
    AND (i1.city != i2.city or i1.state != i2.state)
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(c1).df()
df

,zip,city,city_1,state,state_1
0,60610,CCHICAGO,CHESTNUT STREET,IL,IL
1,60153,MAYWOOD,Maywood,IL,IL
2,60618,Chicago,CCHICAGO,IL,IL
3,60606,312CHICAGO,Chicago,IL,IL
4,60618,Chicago,CCHICAGO,IL,IL
...,...,...,...,...,...
3699,60642,CCHICAGO,Chicago,IL,IL
3700,60642,CCHICAGO,Chicago,IL,IL
3701,60642,CCHICAGO,Chicago,IL,IL
3702,60642,CCHICAGO,Chicago,IL,IL


This query returns rows where the ZIP Code matches between records but the city or state does not.  We see a few instances of case differences, which could be corrected by consistently using upper or lower case.  We also see what appear to be typos in the data that need to be corrected!

#### DBAName -> Zip

We also expect that a given business name should determine its location:

In [22]:
c2 = """
    SELECT i1.DBA_Name, i1.zip, i2.zip 
    FROM inspections i1, inspections i2
    WHERE i1.DBA_Name = i2.DBA_Name
    AND i1.zip != i2.zip
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(c2).df()

df

,dba_name,zip,zip_1
0,LA COCINITA FOOD TRUCK CHICAGO,60201,60077
1,ABUNDANT RESTAURANT,60409,60429
2,PIKO STREET KITCHEN,60076,60193
3,LA COCINITA FOOD TRUCK CHICAGO,60201,60077
4,LA COCINITA FOOD TRUCK CHICAGO,60201,60077
...,...,...,...
61,PIKO STREET KITCHEN,60076,60193
62,PIKO STREET KITCHEN,60076,60193
63,LA COCINITA FOOD TRUCK CHICAGO,60077,60201
64,LA COCINITA FOOD TRUCK CHICAGO,60077,60201


We see several potential issues here, but the cause is unclear. Perhaps a restaurant moved between inspection dates? Perhaps food trucks are inspected at different locations? Let's include the inspection date and address:

(Perhaps DBAName, Inspection Date -> Zip?)

In [23]:
c3 = """
    SELECT i1.DBA_Name, i1.inspection_date, i2.inspection_date, i1.address, i1.address, i1.zip, i2.zip 
    FROM inspections i1, inspections i2
    WHERE i1.DBA_Name = i2.DBA_Name
    AND i1.zip != i2.zip
"""
with duckdb.connect("inspections.db") as con:
    df = con.sql(c3).df()

df

,dba_name,inspection_date,inspection_date_1,address,address_1,zip,zip_1
0,LA COCINITA FOOD TRUCK CHICAGO,2017-06-10,2015-05-06,1625 CHICAGO AVE,1625 CHICAGO AVE,60201,60077
1,ABUNDANT RESTAURANT,2017-02-21,2010-06-30,1509 SIBLEY BLVD,1509 SIBLEY BLVD,60409,60429
2,PIKO STREET KITCHEN,2017-02-07,2014-09-09,3410 MAIN ST,3410 MAIN ST,60076,60193
3,LA COCINITA FOOD TRUCK CHICAGO,2017-01-24,2015-05-06,1625 CHICAGO AVE,1625 CHICAGO AVE,60201,60077
4,LA COCINITA FOOD TRUCK CHICAGO,2017-01-18,2015-05-06,1625 CHICAGO AVE,1625 CHICAGO AVE,60201,60077
...,...,...,...,...,...,...,...
61,PIKO STREET KITCHEN,2016-09-27,2017-02-07,3410 MAIN ST,3410 MAIN ST,60076,60193
62,PIKO STREET KITCHEN,2016-09-20,2017-02-07,3410 MAIN ST,3410 MAIN ST,60076,60193
63,LA COCINITA FOOD TRUCK CHICAGO,2015-05-06,2017-01-24,7770 FRONTAGE RD,7770 FRONTAGE RD,60077,60201
64,LA COCINITA FOOD TRUCK CHICAGO,2016-12-08,2017-06-10,7770 FRONTAGE RD,7770 FRONTAGE RD,60077,60201


Here we can see that the addresses are the same but the ZIP Codes differ, suggesting an error in the data!

## Summary

The relational model is a powerful tool for checking the consistency of data.
In the examples above, we have seen cases where we might need to go back to an earlier phase in the data cleaning workflow, and do more data cleaning in OpenRefine!
We also saw that DuckDB has convenient and powerful ways to explore CSV data. Much more can be done with SQL engines such as DuckDB.
...